In [2]:
from collections import OrderedDict, defaultdict
from typing import Dict
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision.models as models

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

c:\ProgramData\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # 3*32*32 -> 16*32*32
        x = torch.max_pool2d(x, 2)  # 16*32*32 -> 16*16*16
        x = torch.relu(self.conv2(x))  # 16*16*16 -> 32*16*16
        x = torch.max_pool2d(x, 2)  # 32*16*16 -> 32*8*8
        x = torch.flatten(x, start_dim=1, end_dim=-1)  # 32*8*8 -> 2048
        x = torch.relu(self.fc1(x))  # 2048 -> 128
        x = self.fc2(x)  # 128 -> 10
        return x


class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.base = nn.Sequential(
            OrderedDict(
                conv1=nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),  # 3*32*32 -> 6*28*28
                activation1=nn.ReLU(),
                pool1=nn.AvgPool2d(2),  # 6*28*28 -> 6*14*14
                conv2=nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),  # 6*14*14 -> 16*10*10
                activation2=nn.ReLU(),
                pool2=nn.AvgPool2d(2),  # 16*10*10 -> 16*5*5
                flatten=nn.Flatten(start_dim=1),  # 16*5*5 -> 400
                fc1=nn.Linear(400, 120),  # 400 -> 120
                activation3=nn.ReLU(),
                fc2=nn.Linear(120, 84),  # 120 -> 84
                activation4=nn.ReLU(),
                fc3=nn.Linear(84, 10),  # 84 -> 10
            )
        )

    def forward(self, x):
        return self.base(x)


class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        pretrained_weights = models.ResNet50_Weights.DEFAULT
        resnet: models.ResNet = models.resnet50(weights=pretrained_weights)
        self.base = resnet
        self.classifier = nn.Linear(self.base.fc.in_features, 10)
        self.base.fc = nn.Identity()

    def forward(self, x):
        return self.classifier(self.base(x))


class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        pretrained_weights = models.MobileNet_V3_Small_Weights.DEFAULT
        mobilenet: models.MobileNetV3 = models.mobilenet_v3_small(weights=pretrained_weights)
        self.base = mobilenet
        self.classifier = nn.Linear(self.base.classifier[-1].in_features, 10)
        self.base.classifier[-1] = nn.Identity()

    def forward(self, x):
        return self.classifier(self.base(x))

In [4]:
_args = defaultdict(
    model='simple',
    dataset='cifar10',
    epochs=10,
    batch_size=32,
    lr=0.01,
)

model_dict = {
    'simple': SimpleNet,
    'lenet5': LeNet5,
    'resnet50': ResNet,
    'mobilenet': MobileNet,
}
dataset_dict = {
    'cifar10': {
        'loader': datasets.CIFAR10,
        'mean': (0.4914, 0.4822, 0.4465),
        'std': (0.247, 0.243, 0.261),
    },
    'mnist': {
        'loader': datasets.MNIST,
        'mean': (0.1307,),
        'std': (0.3081,),
    },
}

In [5]:
def get_model(device=torch.device('cuda'),
              _args=Dict,
              ):
    if _args['model'] in model_dict:
        model = model_dict[_args['model']]().to(device)
    else:
        raise ValueError(f'model {_args["model"]} is not supported')

    return model


def get_data(_args=Dict,
             is_train=True,
             ):
    if _args['dataset'] in dataset_dict.keys():
        temp_data = dataset_dict[_args['dataset']]
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(temp_data['mean'], temp_data['std']),
        ])
        dataset = temp_data['loader'](str(Path().parent / 'data'),
                                      train=is_train, download=True, transform=transform)
    else:
        raise ValueError(f'dataset {_args["dataset"]} is not supported')

    data_loader = DataLoader(dataset, batch_size=_args['batch_size'], shuffle=True, num_workers=4)

    return data_loader

In [6]:
def run(model,
        dataloader,
        loss_fn,
        optimizer=None,
        is_train=True,
        device=torch.device('cuda'),
        ):
    total_loss, correct = 0.0, 0
    data_len = len(dataloader.dataset)
    model.train() if is_train else model.eval()

    with torch.set_grad_enabled(is_train):
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)

            if is_train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_loss += loss.item()

            pred = output.argmax(dim=1, keepdim=True)  # 확률이 가장 높은 class를 예측값으로 선택
            correct += pred.eq(target.view_as(pred)).sum().item()

    avg_loss = total_loss / data_len
    accuracy = 100. * correct / data_len

    return avg_loss, accuracy

In [4]:
torch.get_num_threads()

AttributeError: module 'torch' has no attribute 'get_num_thread'

In [7]:
# Environments configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model selection
model = get_model(device, _args)
# loss function
loss_fn = torch.nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=_args['lr'])

# Model training
train_loader = get_data(_args, is_train=True)
for epoch in tqdm(range(_args['epochs']), total=_args['epochs'], unit='epoch', desc='Training'):
    train_loss, train_acc = run(model, train_loader, loss_fn, optimizer, device=device, is_train=True)
    tqdm.write(f'Epoch {epoch + 1}/{_args["epochs"]}: Loss: {train_loss:.2f}, Acc: {train_acc:.2f}%')

# Model test
test_loader = get_data(_args, is_train=False)
test_loss, test_acc = run(model, test_loader, loss_fn, device=device, is_train=False)
print(f'Test Loss: {test_loss:.2f}, Test Acc: {test_acc:.2f}%')

Files already downloaded and verified


Training:   0%|          | 0/10 [00:00<?, ?epoch/s]

Epoch 1/10: Loss: 0.05, Acc: 38.47%
Epoch 2/10: Loss: 0.04, Acc: 52.26%
Epoch 3/10: Loss: 0.04, Acc: 58.35%
Epoch 4/10: Loss: 0.03, Acc: 63.02%
Epoch 5/10: Loss: 0.03, Acc: 66.76%
Epoch 6/10: Loss: 0.03, Acc: 69.71%
Epoch 7/10: Loss: 0.02, Acc: 72.47%
Epoch 8/10: Loss: 0.02, Acc: 74.60%
Epoch 9/10: Loss: 0.02, Acc: 76.58%
Epoch 10/10: Loss: 0.02, Acc: 78.54%
Files already downloaded and verified
Test Loss: 0.03, Test Acc: 70.24%
